<a href="https://colab.research.google.com/github/Shreyas2409/Face-Recognition/blob/master/ML_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```
Test data -
https://drive.google.com/drive/folders/1ZVVk4RyGaOHm_Q87YIthSCcG_P6zS5Es?usp=drive_link
 we have made few modifications to train the model pls use this to run it

Train Data - /content/drive/MyDrive/COEN240/train

Test Data  - /content/drive/MyDrive/COEN240/faces (model training)

Test Data - /content/drive/MyDrive/COEN240/test

Test Labels - /content/drive/MyDrive/COEN240/test/labels.txt

PCA With SVM

In [ ]:
import os
import numpy as np
import cv2
import pickle
import sys

from google.colab import drive
drive.mount('/content/drive')







Mounted at /content/drive


In [ ]:
Train = "/content/drive/MyDrive/COEN240/train" # Training dataset
Test = "/content/drive/MyDrive/COEN240/faces"  # Modified Test dataset used for model training
Test2 = "/content/drive/MyDrive/COEN240/test" # Test data to evaluate the model
Labels = "/content/drive/MyDrive/COEN240/test/labels.txt" # Labels of Test data to evaluate the model after training
categories = os.listdir(Train)
categories.remove('.DS_Store')
data_train = []
data_test = []


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/COEN240/train'

In [ ]:
def crop_face(image):


    # Zooming
    zoom_scale = 1.4
    image = cv2.resize(image, None, fx=zoom_scale, fy=zoom_scale, interpolation=cv2.INTER_LINEAR)

     # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Perform face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))


    for (x, y, w, h) in faces:


        face = gray[y:y+h, x:x+w]


        # Apply histogram equalization
        face = cv2.equalizeHist(face)

        # Resize the face image to a fixed size
        face = cv2.resize(face, (50, 50))

        # Normalize the pixel values to [0, 1]
        face = face.astype(np.float32) / 255.0

        face = face.flatten()

        return face


In [ ]:
data = []

for category in categories:
        path = os.path.join(Test, category)
        try:
            label = categories.index(category)
            for img in os.listdir(path):
                imgpath = os. path.join(path, img)
                image = cv2.imread(imgpath)
                if image is not None:
                    image_pre = crop_face(image)
                    if image_pre is not None:
                        data_test.append([image_pre,label])
        except Exception as e:
            print(e)

            print(data_test)

for category in categories:
        path = os.path.join(Train, category)
        try :
            label = categories.index(category)
            for img in os.listdir(path):
                imgpath = os. path.join(path, img)
                image = cv2.imread(imgpath)
                if image is not None:
                    image_pre = crop_face(image)
                    if image_pre is not None:
                        data_train.append([image_pre,label])
        except Exception as e:
            print(e)

In [ ]:

pick_in = open('face_train.pickle', 'wb')
pickle.dump(data_train, pick_in)
pick_in.close()


pick_in = open('face_test.pickle', 'wb')
pickle.dump(data_test, pick_in)
pick_in.close()


In [ ]:
import pickle
import numpy as np
import pandas as pd
import random
import os
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, recall_score
import random

pick_in = open('face_train.pickle', 'rb')
data_train = pickle.load(pick_in)
pick_in.close()

pick_in = open('face_test.pickle', 'rb')
data_test = pickle.load(pick_in)
pick_in.close()



categories = os.listdir(Train)
categories.remove('.DS_Store')






In [ ]:
xtrain = []
ytrain = []
xtest = []
ytest = []

feature =[]
labels = []


for i in data_train:

    xtrain.append(i[0])
    ytrain.append(i[1])

for i in data_test:

    xtest.append(i[0])
    ytest.append(i[1])



# For dimensionality reduction

pca = PCA(n_components=120)
xtrain = pca.fit_transform(xtrain)
xtest = pca.transform(xtest)

scaler = StandardScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

exp_var = sum(pca.explained_variance_ratio_ * 100)
print('Variance explained:', exp_var)


model = SVC(C=20, gamma=0.002,kernel='rbf',class_weight='balanced')
model.fit(xtrain,ytrain)

y_pred = model.predict(xtest)
print('Accuracy: %.3f' % accuracy_score(ytest, y_pred))

# Calculate recall
recall = recall_score(ytest, y_pred,average='micro')
print(recall)


# #Save the model
with open('model_svm_rbf.pkl', 'wb') as f:
    pickle.dump(model, f)

# Save the model
with open('model_pca.pkl', 'wb') as f:
    pickle.dump(pca, f)

with open('model_scalar_standard.pkl', 'wb') as f:
    pickle.dump(scaler, f)













In [ ]:
import pickle
import cv2
import os
import numpy as np
from sklearn.preprocessing import StandardScaler



# Load the saved model
with open('model_svm_rbf.pkl', 'rb') as f:
    model = pickle.load(f)

# Load the saved model
with open('model_pca.pkl', 'rb') as t:
    pca = pickle.load(t)

# Load the saved model
with open('model_scalar_standard.pkl', 'rb') as t:
    scaler = pickle.load(t)










In [ ]:
import shutil
def label_parse(Labelsfile):
  labels = {}
  with open(Labels, 'r') as file:
        for line in file:
            line = line.strip().split()
            image_name, label = line[0], line[1]
            labels[image_name] = label
  return labels

def copy_original_images(Test2, output_folder):
    image_folder = os.path.join(Test2)
    label_file_path = os.path.join(output_folder, 'label.txt')


    labels =label_parse(Labels)
    print(labels)

    os.makedirs(output_folder, exist_ok=True)


    for image_name, label in labels.items():
            image_path = os.path.join(image_folder, image_name)
            output_label_folder = os.path.join(output_folder, label)
            os.makedirs(output_label_folder, exist_ok=True)
            output_path = os.path.join(output_label_folder, image_name)

            if os.path.exists(image_path):
              shutil.copy(image_path, output_path)
            else:
                print(f"Warning: Source file not found at {image_path}")

output_folder = "/content/output_folder"

copy_original_images(Test2, output_folder)




In [ ]:
dataset = '/content/output_folder'


In [ ]:
data = []
for category in categories:
        path = os.path.join(dataset, category)
        try :
            label = categories.index(category)
            for img in os.listdir(path):
                imgpath = os. path.join(path, img)
                image = cv2.imread(imgpath)
                if image is not None:
                    image_pre = crop_face(image)
                    if image_pre is not None:
                        data.append([image_pre,label])
        except Exception as e:
            print(e)

xtest = []
ytest = []
for i in data:
    xtest.append(i[0])
    ytest.append(i[1])

X_scaled_train = pca.transform(xtest)


X_scaled_train = scaler.transform(X_scaled_train)


y_pred = model.predict(X_scaled_train)

print('Accuracy: %.3f' % accuracy_score(ytest, y_pred))



RandomForestClassifier

In [ ]:

import os
import numpy as np
import cv2
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline

# Load data
pick_in = open('face_train.pickle', 'rb')
data_train = pickle.load(pick_in)
pick_in.close()

pick_in = open('face_test.pickle', 'rb')
data_test = pickle.load(pick_in)
pick_in.close()

# Prepare data
xtrain, ytrain = zip(*data_train)
xtest, ytest = zip(*data_test)

# For dimensionality reduction
pca = PCA(n_components=120)

# Feature scaling
scaler = StandardScaler()

# Random Forest model
rf_model = RandomForestClassifier(random_state=52)

# Create a pipeline with PCA, scaling, and Random Forest
pipeline = Pipeline([
    ('pca', pca),
    ('scaler', scaler),
    ('rf', rf_model)
])

# Hyperparameter tuning
param_grid = {
    'pca__n_components': [80, 100, 120],
    'rf__n_estimators': [100, 150, 200],
    'rf__max_depth': [None, 10, 20],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='accuracy')
grid_search.fit(xtrain, ytrain)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Train Random Forest model with the best hyperparameters
best_pipeline = grid_search.best_estimator_
best_pipeline.fit(xtrain, ytrain)

# Predictions on the test set
y_pred = best_pipeline.predict(xtest)

# Evaluate the model
test_accuracy = accuracy_score(ytest, y_pred)
f1 = f1_score(ytest, y_pred, average='micro')
recall = recall_score(ytest, y_pred, average='micro')

print(f'Test Accuracy: {test_accuracy}')
print(f'F1 Score: {f1}')
print(f'Recall: {recall}')

# Save the best Random Forest model
with open('model_random_forest.pkl', 'wb') as f:
    pickle.dump(best_pipeline, f)

# Save PCA and scaler for future use
with open('model_pca.pkl', 'wb') as f:
    pickle.dump(pca, f)

with open('model_scalar_standard.pkl', 'wb') as f:
    pickle.dump(scaler, f)


GridSearch SVM

In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma' : [0.0001, 0.001, 0.1,1],
    'kernel' : ['rbf']
}

svc = SVC(probability=True)


model = GridSearchCV(svc,param_grid,cv=4)
model.fit(xtrain, ytrain)


print(model.best_params_)

y_pred = model.predict(xtest)
print(f"accuracy  {accuracy_score(y_pred,ytest) * 100}% ")

Stochastic gradient descent

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(), SGDClassifier(loss='log_loss'))


model.fit(xtrain, ytrain)


y_pred = model.predict(xtest)


accuracy = np.mean(y_pred == ytest)
print('Accuracy:', accuracy)
